# GLOBAL UNDERNUTRITION STUDY - EXPLORATION  

*BY FURAWA*  

**Table of Contents**  

1. [Data collection](#data_collection)  
2. [Data discovery](#data_discovery)  
3. [Data cleaning](#data_cleaning)  
4. [Computing new variables to lead the analysis](#new_variables)  
5. [Identify major trends](#major_trends)  

In [1]:
# Import all the needed libraries for the project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import re
from pandas.api.types import CategoricalDtype

%matplotlib inline
pd.set_option('max_rows', 20)

<a id='data_collection'></a>
## 1. Data collection  
All the data has been downloaded from the [FAO](http://www.fao.org/faostat/en/#data) website.  
Let us check the files.

In [2]:
# Store the file names in the file_names variable
file_names = glob('files/*.csv')
# Check the file_names list
file_names

['files/food_balance_cereals.csv',
 'files/food_security_indicators.csv',
 'files/food_balance_vegetal.csv',
 'files/food_balance_animal.csv',
 'files/population.csv']

In [3]:
# Loop into the file_names list
for file in file_names:
    # Read each file in the file_names list and assign it to a variable retrieved from the file name
     exec(re.split('\. |\W', file)[1] + "=  pd.read_csv(file)")

In [4]:
# Check the dataframe info
food_security_indicators.head(2)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,7.9,F,FAO estimate,NaN
1,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20132015,2013-2015,millions,8.8,F,FAO estimate,NaN


In [5]:
# Check the dataframe info
food_balance_vegetal.head(2)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2511,Wheat and products,2013,2013,1000 tonnes,5169.0,S,Standardized data
1,FBS,Food Balance Sheets,2,Afghanistan,5611,Import Quantity,2511,Wheat and products,2013,2013,1000 tonnes,1173.0,S,Standardized data


In [6]:
# Check the dataframe info
food_balance_animal.head(2)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,2013,2013,1000 tonnes,134.0,S,Standardized data
1,FBS,Food Balance Sheets,2,Afghanistan,5611,Import Quantity,2731,Bovine Meat,2013,2013,1000 tonnes,6.0,S,Standardized data


In [7]:
# Check the dataframe info
food_balance_cereals.head(2)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2511,Wheat and products,2013,2013,1000 tonnes,5169.0,S,Standardized data
1,FBS,Food Balance Sheets,2,Afghanistan,5611,Import Quantity,2511,Wheat and products,2013,2013,1000 tonnes,1173.0,S,Standardized data


In [8]:
population.head(2)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,511,Total Population - Both sexes,2501,Population,1961,1961,1000 persons,8954.0,NaN,Official data
1,FBS,Food Balance Sheets,2,Afghanistan,511,Total Population - Both sexes,2501,Population,1962,1962,1000 persons,9142.0,NaN,Official data


Except from the food_security_indicators dataframe all the other dataframes have the same 14 columns.  

<a id='data_discovery'></a>
## 2. Data Discovery

Let us check the primary key of each table and test them.  
We will create a function to find the primary key.  

In [9]:
# Function to find the potential primary keys
def check_potential_primary_key(df) -> str:
    # Loop in the column list of the specific dataframe
    for column_pk in df.keys():
        # Remove the duplicated values from each column and check if the size is the same as the df
        if len(df) != len(df[column_pk].drop_duplicates()):
            # No output if the column is not a primary key
           None
        else:
            # Print all the potential primary keys
            print("{} could be a primary key!".format(column_pk))

Now we can use the function to find the potential primary keys of each dataframe. 

In [11]:
# Check the primary key of population dataframe
check_potential_primary_key(population)

There are no potential primary key in the dataframe, due to the fact that we have the same countries and countries code repeating for each year.

In [12]:
# Check the primary key of food balance vegetal
check_potential_primary_key(food_balance_vegetal)

We have no output, which means there are no potential primary key in this dataframe.

In [13]:
# Check the primary key of food balance livestock
check_potential_primary_key(food_balance_animal)

Same here, there are no primary keys in the food balance livestock

In [14]:
# Check the primary key of food balance cereals
check_potential_primary_key(food_balance_cereals)

In [15]:
# Check the primary key of food security indicators
check_potential_primary_key(food_security_indicators)

Even for the food balance cereals and food security indicators dataframes we have no potential primary keys

Let us create column with the total population and remove some useless columns from the population dataframe.

In [18]:
# Create the population column, we retrieve the 1000 in the Unit column and multiply it by the Value column
population["population"] = int(population.Unit.str.split(" ")[0][0]) * population.Value
# Remove some useless columns
population_df = population.drop(population.columns.difference(["Year", "Country Code", "Country", "population"]),
                                axis =1)
# Check the dataframe
population_df.head()

,Country Code,Country,Year,population
0,2,Afghanistan,1961,8954000.0
1,2,Afghanistan,1962,9142000.0
2,2,Afghanistan,1963,9340000.0
3,2,Afghanistan,1964,9547000.0
4,2,Afghanistan,1965,9765000.0


Now we can calculate the total numbers of human involved.

In [ ]:
# Calcalute the total number of humans in 2013
total_population = population_df.query"Year".population.sum()
print("The total number of humans on the planet is : {:,}".format(total_population))

This result cannot be correct, mostly if we talk about the 2013 world population. actually in 2019 the world population is around 7.7 billion. There must be an error, we will go deep to check the issue.  

<a id='data_cleaning'></a>
## 3. Data Cleaning  

The dataframes are downloaded and loaded but dirty. There are useless rows and columns, anomalies in the population data must be corrected, the columns names must be changed. Let's do some cleaning.  
We start putting all the food balance dataframes in one unique dataframe.  
- **Food balance**

In [ ]:
# Create the origin variable in each balance food dataframe to store the food origin
food_balance_animal["origin"] = "animal"
food_balance_cereals["origin"] = "cereal"
food_balance_vegetal["origin"] = "vegetal"

In [ ]:
# Append the 3 dataframes in one unique dataframe
food_balance_df = food_balance_animal.append(food_balance_vegetal)
# Check the first rows
food_balance_df.shape

In [ ]:
# Select the rows where Unit equal 1000 tonnes and convert the respectives values
food_balance_df.loc[food_balance_df.Unit == "1000 tonnes", "Value"] *= 1000000

In [ ]:
food_balance_df

In [ ]:
# Delete the 3 useless balance food dataframe
del food_balance_animal, food_balance_vegetal

In [ ]:
# Rename the columns
food_balance_df.rename(columns = {"Country Code":"country_code", "Country":"country", "Element":"element",
                                  "Item Code":"item_code", "Item":"item", "Year":"year",
                                  "Value":"value"}, inplace = True)

In [ ]:
# Transform the dataframe from long to wide with pivot_table
food_balance_wide = food_balance_df.pivot_table(
    # Put as index the Columns that we want to keep in the dataframe
    index = ["country_code", "country", "item_code", "item", "year", "origin"],
    # Select the columns that we want to transform from long to wide and the values that we sum 
    columns = ["element"], values = ["value"], aggfunc = sum)
# Renaming the columns 
food_balance_wide.columns = ["domestic_supply_quantity", "export_quantity", "fat_supply_quantity_gcapitaday",
                             "feed", "food", "food_supply_kcalcapitaday", "food_supply_quantity_kgcapitayr", 
                            "import_quantity", "losses", "other_uses", "processing", "production", 
                            "protein_supply_quantity_gcapitaday", "seed", "stock_variation"]

In [ ]:
# Reset the index to have normal columns
food_balance = food_balance_wide.reset_index()
# delete the useless dataframe
del food_balance_df, food_balance_wide
# Check the first rows of the dataframe
food_balance.head()

Looking at the definitions of the elements in the [FAO](http://www.fao.org/faostat/en/#data/FBS)(Definitions and Standards tab), we notice that there are redundant information concerning those elements. Let's indentify the redundancy with a mathematical formula and give and example with the wheat in France.


In [ ]:
# Create a dataframe with France as country and wheat as item
wheat_france = food_balance.query("country == 'France' and item == 'Wheat and products'")
# Formulas
print("Formula 1 : Domestic supply = Production + Import Quantity + Stock Variation - Export Quantity \n\
Formula 2 : Domestic supply = Food + Feed + Losses + Seed + Processing + Other Uses")

In [ ]:
# Apply the formula in the wheat france dataframe
term_1 = (wheat_france[:1].production + wheat_france[:1].import_quantity + wheat_france[:1].stock_variation \
         - wheat_france[:1].export_quantity).values[0]

term_2 = wheat_france[:1].domestic_supply_quantity.values[0]

term_3 = (wheat_france[:1].food + wheat_france[:1].feed + wheat_france[:1].losses + wheat_france[:1].seed + \
         wheat_france[:1].processing + wheat_france[:1].other_uses).values[0]

In [ ]:
# Check if all the term are equal, no output means it is correct
assert term_1 == term_2 == term_3

In [ ]:
print("For the wheat in France we have : \n\
Domestic supply quantity = {:,} kg \n\
Production + Import Quantity + Stock Variation - Export Quantity = {:,} kg \n\
Food + Feed + Losses + Seed + Processing + Other Uses = {} kg".format(term_2, term_1, term_3))

- **Food Security Indicators**

In [ ]:
food_security_indicators.info()

- The Value column has 605 non-null rows which means that there are many null values  
- There are many useless columns to remove  
- We need to change the columns name
- The Value must be float number not a string  
- The year must be a categorical variable not a string 
- There is a value <0.1, that must be changed  
- Convert the value in million

Let us make all those changes.

In [ ]:
food_security_indicators[food_security_indicators.Value.isnull()][:5]

All these rows with NaN values are useless. We will remove them. 

In [ ]:
# Remove all NaN rows from the dataframe
food_security_indicators = food_security_indicators[food_security_indicators.Value.isnull() == False]

In [ ]:
# Check that the Value column has no more NaN values, no output means it is correct
assert food_security_indicators.Value.isnull().all() == False

Now we can change the columns name and keep the columns that we need for the Analysis.

In [ ]:
# Select the columns that we need
indicators_df = food_security_indicators.loc[:, ["Area Code", "Area", "Year", "Value"]]
# Change the columns name
indicators_df.columns = ["country_code", "country", "year", "value"]
# Turn the year variable into a categorical type
category_type = CategoricalDtype(categories = ["2012-2014", "2013-2015", "2014-2016", "2015-2017", "2016-2018"]
                                               ,ordered = True)
indicators_df.year = indicators_df.year.astype(category_type)
indicators_df.year = indicators_df.year.cat.rename_categories([2013, 2014, 2015, 2016, 2017])

In [ ]:
# Assert that the data type is correct
assert indicators_df.year.dtype == "category"

In [ ]:
# Replace the <0.1 value with 0.09 
indicators_df.value = indicators_df.value.apply(lambda x : x.replace("<0.1", "0.09"))

In [ ]:
# Check if there is no <0.1 value anymore
assert indicators_df.loc[indicators_df.value == "<0.1"].value.any() == False

In [ ]:
# Change the data type of the value columns from string to float
indicators_df.value = indicators_df.value.astype("float64")

In [ ]:
# Assert that the changes occurs
assert indicators_df.value.dtype == "float64"

In [ ]:
indicators_df.info()

In [ ]:
indicators_df.sample(3)

In [ ]:
# Convert the value variable in million 
indicators_df.value = indicators_df.value * 1000000
indicators_df.head(2)

- **Population**  

There is one flag value. Let see what it is about

In [ ]:
population[population.Flag.isnull() == False]

We see here that The "China" entry as country is an aggregate value. Let's check if there are another entry with China.

In [ ]:
# Find all the entries with china
population_df[population_df.Country.str.contains("China")]

In [ ]:
# Sum the four first values
population_df.population[32:36].sum()

We can see that China, Hong Kong SAR, China, Macao SAR, China, mainland and China, Taiwan Province of are actually four parts of the China country. We can confirm this because the total population of the four parts are equal to the population of the China country.
We will remove the China population value from the dataframe during the calculation of the total number of humans on the planet.

In [ ]:
# Remove China from the dataframe
population_df = population_df[population_df.Country != "China"]

In [ ]:
# Compute the wold population
world_population = population_df.population.sum()
print("The total number of humans on the planet in 2013 is : {:,}".format(world_population))

In [ ]:
# Rename the columns of the dataframe
population_df.columns = ["country_code", "country", "population"]

In [ ]:
population_df.info()

<a id=new_variables></a>
## 4. Computing New Variables To Lead the Analysis 

All the dataframes are clean, we can compute some new variables for future analysis

- **food_supply_kcal(food supply express in kcal)**

In [ ]:
# Create a temporary dataframe where we join the population and the food_balande dfs together 
food_balance_full = pd.merge(population_df, food_balance, how = "left", on = ["country", "country_code"])
food_balance_full.shape

In [ ]:
# Compute food_supply_kcal by multiplying by the population of each country and by 365 days
food_balance_full["food_supply_kcal"] = food_balance_full.food_supply_kcalcapitaday * food_balance_full.population * 365
food_balance_full.head(2)

- **food_supply_kgprotein(food supply express in kg protein)**

In [ ]:
# Compute food_supply_kgprotein, dividing by 1000 to have it in kg then multiplying by 365 for 1 year
food_balance_full["food_supply_kgprotein"] = (food_balance_full.protein_supply_quantity_gcapitaday / 1000) * food_balance_full.population * 365
food_balance_full.head(2)

- **food_supply_kg(food supply express in kg)**

In [ ]:
# Compute food_supply_kg just multiplying food by 1 million  
food_balance_full["food_supply_kg"] = food_balance_full.food
food_balance_full.head(2)

- **ratio_kcalkg(energy:weight ratio of each)**  
This variable is computed using food_supply_kcal and food_supply_kg

In [ ]:
# Compute ratio_kcalkg (food_supply_kcal / food_supply_kg)  
food_balance_full["ratio_kcalkg"] = round(food_balance_full.food_supply_kcal / food_balance_full.food_supply_kg,2)
food_balance_full.ratio_kcalkg = food_balance_full.ratio_kcalkg.replace((np.inf, -np.inf), 0)
food_balance_full.head(2)

In [ ]:
# Check the ratio of an italian egg
food_balance_full.query("item == 'Eggs' and country == 'Italy'").ratio_kcalkg

- **protein_percentage(pretein percentage of each item)**

In [ ]:
food_balance_full["protein_percentage"] = round(food_balance_full.food_supply_kgprotein / food_balance_full.food_supply_kg * 100,1)
food_balance_full.protein_percentage = food_balance_full.protein_percentage.replace((np.inf, -np.inf), 0)
food_balance_full.head(2)

In [ ]:
food_balance_full.query("item == 'Eggs' and country == 'Italy'").protein_percentage

According to [wikipedia](https://en.wikipedia.org/wiki/Egg_as_food#Nutritional_value) 50g of egg provides approximately 70 calories  and 6g of protein.   
In italy we have a value of 11.4 percent of protein in one egg which means that in 50g we will have $50 * 11.4 / 100 = 5.7g$ of protein which is almost the same as the value of wikipedia.

- **dom_sup_kcal(Global domestic supply in kcal)**

In [ ]:
# Compute dom_sup_kcal 
food_balance_full["dom_sup_kcal"] = food_balance_full.domestic_supply_quantity * food_balance_full.ratio_kcalkg
food_balance_full.head(2)

- **dom_sup_kgprot(Global domestic supply in kg of protein)**

In [ ]:
# Compute dom_sup_kgprot variable 
food_balance_full["dom_sup_kgprot"] = food_balance_full.domestic_supply_quantity * food_balance_full.protein_percentage/100
food_balance_full.head(2)

- **great_import_from_undern_countries (200 highest imports of the 25 most exported items from countries with more than 10% malnourishment)**      
The variable must be a boolean(True or False)  
    1. Countries with a malnourishment rate of more than 10%

In [ ]:
# Merge indicators_df and population_df toghether
ind_pop = indicators_df.merge(population_df, how = "left", on = ["country_code", "country"])
# Compute the ratio
ind_pop["ratio"] = round(ind_pop.value / ind_pop.population * 100, 1)
# Find the countries with ratio greater than 10% in 2013 
ind_pop_10 = ind_pop.query("ratio > 10 & year == 2013")
ind_pop_10.shape

There are 76 countries with a manoulrishment rate of more than 10%.

    2. 25 most exported items(in terms of quantity) by these countries for any given year.  

In [ ]:
# Merge ind_pop and food_balance_full together to get the export quantity and items
exported_items_25 = ind_pop_10.merge(food_balance_full, how = "left"). \
sort_values(by = "export_quantity", ascending = False).loc[:,("item")].drop_duplicates()[:25]
# Print the 25 items from the highest to the lowest
for i, item in enumerate(exported_items_25):
    print("{} - {}".format(i+1, item))

    3. 200 highest import quantities among the 25 items

In [ ]:
# Select the 200 highest import quantities
high_imp_200 = food_balance_full[food_balance_full.item.isin(exported_items_25.tolist())] \
.sort_values(by = "import_quantity", ascending = False)[:200]
high_imp_200.shape

    4. Create great_import_from_undern_countries variable and set it to True for the 200 higest importation quantities

In [ ]:
# If the index of food_balance_full is in high_imp_200 we set the variable to True othewise False
food_balance_full["great_import_from_undern_countries"] = food_balance_full.index.isin(high_imp_200.index)
# Check if there are just 200 lines with True for the variable
print(food_balance_full[food_balance_full.great_import_from_undern_countries == True].shape)
food_balance_full.head(2)

<a id="major_trends"></a>
## 5. Identify Major Trends  
### 5.1 Proportion of the global domestic supply considering only plant products  
- **Proportion used as food**

In [ ]:
# Create a dataframe with just vegetal products
vegetal_full = food_balance_full.query("origin == 'vegetal'")
# Compute the global domestic supply 
global_domestic_supply = vegetal_full.domestic_supply_quantity.sum()
global_domestic_supply

In 2013 the global domestic supply is about **8,482,244,000,000** kg

In [ ]:
# Compute the proportion of domestic supply used as food
vegetal_food_prop = round(vegetal_full.food_supply_kg.sum() / global_domestic_supply * 100, 2)
vegetal_food_prop

Just 43.53% of the global domestic food is actually used as food

- **Proportion used as feed**

In [ ]:
vegetal_feed_prop = round(vegetal_full.feed.sum() / global_domestic_supply * 100, 2)
vegetal_feed_prop

- **Proportion used as waste**

In [ ]:
vegetal_waste_prop = round(vegetal_full.losses.sum() / global_domestic_supply * 100, 2)
vegetal_waste_prop

- **Proportion used as other uses**

In [ ]:
vegetal_other_prop = round(vegetal_full.other_uses.sum() / global_domestic_supply * 100, 2)
vegetal_other_prop

### 5.2 Number of humans that could be fed with vegetal  
We will give the result in term of calories and protein and express it as a percentage of the world's population.
It is difficult to know the number of calories we need to survive because it depends on the age, weight, activity level and gender. For long term health most people will need a minimum of 1200 calories per day. 
Let's take according to the [Dietary Guidelines](https://health.gov/dietaryguidelines/2015/guidelines/appendix-2/) an average of 2400 calories per day per human.  
- **Result considering Kcal**  

In [ ]:
# Compute the vegetal global food supply in kcal
vegetal_global_supply_kcal = vegetal_full.dom_sup_kcal.sum()
# Compute food supply in kcal considering just food and feed proportions
vegetal_food_supply_kcal = vegetal_global_supply_kcal * (vegetal_food_prop + vegetal_feed_prop) / 100
# Compute the number of humans that could be fed Considering 2400 calories per day
vegetal_humans_fed_kcal = vegetal_food_supply_kcal / 2400 /365
# Percentage of the world population
percentage_fed_kcal = round(vegetal_humans_fed_kcal / world_population * 100)
percentage_fed_kcal

- **Result considering kgprot**

In [ ]:
# Compute the vegetal global food supply in kprot
vegetal_global_supply_kgprot = vegetal_full.dom_sup_kgprot.sum()
# Compute food supply in kcal considering just food and feed proportions
vegetal_food_supply_kgprot = vegetal_global_supply_kgprot * (vegetal_food_prop + vegetal_feed_prop) / 100
# Compute the number of humans that could be fed Considering 56 grams per day
vegetal_humans_fed_kgprot = vegetal_food_supply_kgprot / 0.056 / 365
# Percentage of the world population
percentage_fed_kgprot = round(vegetal_humans_fed_kgprot / world_population)
percentage_fed_kgprot

With just the vegetal global supply including just the food and the feed, about 116% of the population can be fed
in term of kcal and kg of protein intake.

### 5.3 Number of humans that could be fed with the global food supply  
We consider food supply as variable and not domestic food supply
- **Result in Kcal**


In [ ]:
# Compute the total food supply in kcal
global_food_supply_kcal = food_balance_full.food_supply_kcal.sum()
# Compute number of humans that could be fed considering 2400 kcal per day
humans_fed_kcal = global_food_supply_kcal / 2400 / 365
# Percentage of humans compared to the world population
percentage_humans_fed_kcal = round(humans_fed_kcal / world_population * 100)
percentage_humans_fed_kcal

In [ ]:
# Compute the total food supply in kgprot
global_food_supply_kgprot = food_balance_full.food_supply_kgprotein.sum()
# Compute number of humans that could be fed considering 56 grams of proteins
humans_fed_kgprot = global_food_supply_kgprot / 0.056 / 365
# Percentage of humans compared to the world population
percentage_humans_fed_kgprot = round(humans_fed_kgprot / world_population * 100)
percentage_humans_fed_kgprot

With the global food supply we can feed about 120% of the population considering the daily calories intake, and about 145% of the population considering the daily kg per protein intake.

### 5.4 Proportion of the world population considered undernourished

In [ ]:
# Use the ind_pop already created to create a dataframe with just 2013
ind_pop_2013 = ind_pop.query("year == 2013")
# Total number of humans undernourished
global_undern_pop = ind_pop_2013.value.sum()
# Proportion of humans undernourished, considering the world population
prop_undern = round(global_undern_pop / world_population * 100)
prop_undern

13% of the world population is actually undernourished. 

### 5.5 3 highest exported items
- **Considering the other uses to domestic supply quantity ratio**

In [ ]:
# Compute other uses to domestic supply quantity ratio in food_balance_full
food_balance_full["other_supply_ratio"] = food_balance_full.other_uses / food_balance_full.domestic_supply_quantity
# Compute feed to food+feed ratio in food_balance_full
food_balance_full["feed_foodfeed_ratio"] = food_balance_full.feed / (food_balance_full.feed + food_balance_full.food)
food_balance_full.head(2)

In [ ]:
# Compute the 3 highest exportation considering other_supply_ratio
sup_exp_3 =food_balance_full[food_balance_full.item.isin(exported_items_25.tolist())] \
                .sort_values(by = "other_supply_ratio", ascending = False).item.drop_duplicates()[:3]
sup_exp_3

In [ ]:
# Compute the 3 highest exportation considering feed_foodfeed_ratio
feed_exp_3 = food_balance_full[food_balance_full.item.isin(exported_items_25.tolist())] \
            .sort_values(by = "feed_foodfeed_ratio", ascending = False).item.drop_duplicates()[:3]
feed_exp_3

### 5.6 Proportion of cereals used for feed

In [ ]:
# Retrieve all the cereals items from food_balance_cereals
cereals_items = food_balance_cereals.Item.tolist()
# Create a dataframe with just cereals from food_balance_full
cereals_df = food_balance_full[food_balance_full.item.isin(cereals_items)]
# Proportion used for feed
feed_cereals_prop = round(cereals_df.feed.sum() / (cereals_df.food.sum() + cereals_df.feed.sum()) * 100, 2)
feed_cereals_prop

45.91% of cereals are used for feed.

### 5.7 Tons of cereals that could be release if the US reduce its production of animal products by 10%

In [ ]:
# Create a dataframe with just US
us_cereals = cereals_df.query("country == 'United States of America'").copy()
# Compute 10 percent of total animal feed used for animal products in tonnes
us_feed_10 = us_cereals.feed.sum() * 0.1 / 1000
# Convert in kcal using ratio_kcalkg
us_cereals["feed_kcal"] = us_cereals.feed * us_cereals.ratio_kcalkg
# Find the number of people that could be fed
fed_humans_number = round(us_cereals.feed_kcal.sum() / 2400 / 365 * 0.1)
print("{:,} tonnes of food can be released if the US reduce its production by 10%, that quantity of food can actually feed {:,} people".format(us_feed_10, fed_humans_number))

### 5.8 Proportion of cassava exported and undernutrition in Thailand

- **Proportion of cassava exported**

In [ ]:
# Quantity of cassava exported in Thailand
thai_cassava_exp = food_balance_full.query("country == 'Thailand' & item == 'Cassava and products'").export_quantity / 1000
# Proportion of Undernutrition in thailand
thai_undern = round(ind_pop_2013.query("country == 'Thailand'").ratio.values[0],1)
# total production of cassava in thailande
thai_cassava_prod = food_balance_full.query("country == 'Thailand' & item == 'Cassava and products'").production / 1000
# Proportion of cassava exported
prop_cassava_exp = round((thai_cassava_exp / thai_cassava_prod.values).values[0],2)
print("{}% of cassava is exported in Thailand while {}% of the population are undernourished".format(prop_cassava_exp*100, thai_undern))

## 6 Communicate
### 6.1 Undernutrition over time



In [ ]:
indicators_df.groupby("year").value.sum().plot();

The undernutrition decreases from 2013 to 2015, then started increasing again from 2015 to 2017.

### 6.2 Projected population size in 2050